In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
user_features = df[['person']].drop_duplicates().copy()

## Features "booleanas"

Estas son features del estilo "tiene búsquedas o no".

In [4]:
users_con_vistas = set(df.loc[df['event'] == 'viewed product', 'person'])
user_features['vio_productos'] = user_features['person'].isin(users_con_vistas)

users_con_checkouts = set(df.loc[df['event'] == 'checkout', 'person'])
user_features['tiene_checkouts'] = user_features['person'].isin(users_con_checkouts)

users_con_brand_listing = set(df.loc[df['event'] == 'brand listing', 'person'])
user_features['tiene_brand_listing'] = user_features['person'].isin(users_con_brand_listing)

users_con_generic_listing = set(df.loc[df['event'] == 'generic listing', 'person'])
user_features['tiene_generic_listing'] = user_features['person'].isin(users_con_generic_listing)

users_con_visitas = set(df.loc[df['event'] == 'visited site', 'person'])
user_features['tiene_visitas'] = user_features['person'].isin(users_con_visitas)

users_llegaron_por_ad = set(df.loc[df['event'] == 'ad campaign hit', 'person'])
user_features['llegaron_por_ad'] = user_features['person'].isin(users_llegaron_por_ad)

users_llegaron_por_search = set(df.loc[df['event'] == 'search engine hit', 'person'])
user_features['llegaron_por_search'] = user_features['person'].isin(users_llegaron_por_search)

users_con_leads = set(df.loc[df['event'] == 'lead', 'person'])
user_features['tiene_leads'] = user_features['person'].isin(users_con_leads)

users_con_busquedas = set(df.loc[df['event'] == 'searched products', 'person'])
user_features['tiene_busquedas'] = user_features['person'].isin(users_con_busquedas)

users_con_conversions = set(df.loc[df['event'] == 'conversion', 'person'])
user_features['tiene_conversions'] = user_features['person'].isin(users_con_conversions)
user_features.set_index('person', inplace=True)

In [5]:
user_features.head()

,vio_productos,tiene_checkouts,tiene_brand_listing,tiene_generic_listing,tiene_visitas,llegaron_por_ad,llegaron_por_search,tiene_leads,tiene_busquedas,tiene_conversions
person,,,,,,,,,,
4886f805,True,True,False,True,True,False,True,False,True,False
ad93850f,True,True,True,True,True,True,True,False,False,False
0297fc1e,True,True,True,True,True,True,False,True,True,False
2d681dd8,True,True,True,True,True,True,True,False,True,False
cccea85e,True,True,True,True,True,True,True,False,True,False


## Cantidad de cada evento para cada usuario

In [6]:
def agregar_feature(nombre, evento):
    feature_df = df.loc[df['event'] == evento]
    feature_df = feature_df.groupby('person')['event'].value_counts().unstack()
    feature_df.rename(columns={evento:'cant_'+nombre}, inplace=True)
    return feature_df

In [7]:
conversion = agregar_feature('conversions', 'conversion')
user_features = user_features.join(conversion).fillna(0)

In [8]:
checkout = agregar_feature('checkouts', 'checkout')
user_features = user_features.join(checkout).fillna(0)

In [9]:
viewed_product = agregar_feature('viewed_product', 'viewed product')
user_features = user_features.join(viewed_product).fillna(0)

In [10]:
searched_product = agregar_feature('searched_product', 'searched products')
user_features = user_features.join(searched_product).fillna(0)

In [12]:
visited_site = agregar_feature('visitas', 'visited site')
user_features = user_features.join(visited_site).fillna(0)

In [13]:
lead = agregar_feature('leads', 'lead')
user_features = user_features.join(lead).fillna(0)

In [14]:
brand_listing = agregar_feature('brand_listings', 'brand listing')
user_features = user_features.join(brand_listing).fillna(0)

In [15]:
generic_listing = agregar_feature('generic_listings', 'generic listing')
user_features = user_features.join(generic_listing).fillna(0)

In [16]:
ads = agregar_feature('entradas_ads', 'ad campaign hit')
user_features = user_features.join(ads).fillna(0)

In [17]:
search_engine = agregar_feature('entradas_buscador', 'search engine hit')
user_features = user_features.join(search_engine).fillna(0)

In [18]:
user_features.describe().T

,count,mean,std,min,25%,50%,75%,max
cant_conversions,38829.0,0.182621,0.987561,0.0,0.0,0.0,0.0,129.0
cant_checkouts,38829.0,1.682119,2.717728,0.0,1.0,1.0,2.0,197.0
cant_viewed_product,38829.0,32.144119,79.762927,0.0,3.0,10.0,29.0,2355.0
cant_searched_product,38829.0,3.363878,12.897345,0.0,0.0,0.0,2.0,739.0
cant_visitas,38829.0,5.255582,10.335250,0.0,1.0,2.0,5.0,295.0
cant_leads,38829.0,0.025316,0.349768,0.0,0.0,0.0,0.0,38.0
cant_brand_listings,38829.0,5.570888,23.437437,0.0,0.0,0.0,4.0,1891.0
cant_generic_listings,38829.0,4.125164,10.258573,0.0,0.0,1.0,4.0,470.0
cant_entradas_ads,38829.0,4.928996,10.821833,0.0,1.0,2.0,5.0,523.0
cant_entradas_buscador,38829.0,2.740374,6.878850,0.0,0.0,1.0,3.0,762.0


## Sesiones

In [19]:
sesiones = df.copy()

In [20]:
sesiones.sort_values(by='timestamp', inplace=True)
sesiones['diff'] = sesiones.groupby('person')['timestamp'].diff() / np.timedelta64(1, 'h')
sesiones['diff'].fillna(0, inplace=True)

In [21]:
sesiones[['timestamp', 'person', 'diff']].head(10)

,timestamp,person,diff
2307205,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753202,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753201,2018-01-01 08:09:31,0f4e2a4b,0.000000
1753200,2018-01-01 08:09:44,0f4e2a4b,0.003611
1753207,2018-01-01 08:45:29,0f4e2a4b,0.595833
2307206,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753210,2018-01-01 08:45:29,0f4e2a4b,0.000000
1753203,2018-01-01 08:45:43,0f4e2a4b,0.003889
1753204,2018-01-01 08:48:57,0f4e2a4b,0.053889
1753213,2018-01-01 08:49:05,0f4e2a4b,0.002222


In [22]:
sesiones['new_session'] = sesiones.groupby('person')['diff'].apply(lambda x: x > 1.0)
sesiones['session_id'] = sesiones.groupby('person')['new_session'].cumsum()

In [23]:
sesiones[['timestamp', 'person', 'event', 'diff', 'session_id']].head()

,timestamp,person,event,diff,session_id
2307205,2018-01-01 08:09:31,0f4e2a4b,visited site,0.000000,0.0
1753202,2018-01-01 08:09:31,0f4e2a4b,search engine hit,0.000000,0.0
1753201,2018-01-01 08:09:31,0f4e2a4b,ad campaign hit,0.000000,0.0
1753200,2018-01-01 08:09:44,0f4e2a4b,viewed product,0.003611,0.0
1753207,2018-01-01 08:45:29,0f4e2a4b,ad campaign hit,0.595833,0.0


In [24]:
#cantidad_eventos_usuario = sesiones.loc[sesiones['event'] == 'checkout']
#sesiones = sesiones.loc[sesiones['event'] == 'checkout']

In [25]:
cantidad_eventos_usuario = sesiones.groupby(['person', 'session_id'])['event']\
    .value_counts().unstack().unstack().sum(axis=1).to_frame().rename(columns={0:"total_eventos"})

In [26]:
cantidad_eventos_usuario.head()

,total_eventos
person,
0008ed71,6.0
00091926,448.0
00091a7a,10.0
000ba417,206.0
000c79fe,17.0


In [27]:
sesiones_por_usuario = sesiones.groupby('person')['session_id'].unique().apply(lambda x: len(x)).\
                        to_frame().rename(columns={'session_id':'total_sesiones'})
promedio_eventos_sesion = cantidad_eventos_usuario.join(sesiones_por_usuario)

In [28]:
promedio_eventos_sesion['promedio_eventos_por_sesion'] = promedio_eventos_sesion['total_eventos']\
                                            / promedio_eventos_sesion['total_sesiones']

In [29]:
user_features = user_features.join(promedio_eventos_sesion[['total_sesiones', 'promedio_eventos_por_sesion']])

In [30]:
user_features.fillna(0, inplace=True)

In [31]:
user_features.head()

,vio_productos,tiene_checkouts,tiene_brand_listing,tiene_generic_listing,tiene_visitas,llegaron_por_ad,llegaron_por_search,tiene_leads,tiene_busquedas,tiene_conversions,...,cant_viewed_product,cant_searched_product,cant_visitas,cant_leads,cant_brand_listings,cant_generic_listings,cant_entradas_ads,cant_entradas_buscador,total_sesiones,promedio_eventos_por_sesion
person,,,,,,,,,,,,,,,,,,,,,
4886f805,True,True,False,True,True,False,True,False,True,False,...,4.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1,9.000000
ad93850f,True,True,True,True,True,True,True,False,False,False,...,20.0,0.0,5.0,0.0,15.0,7.0,10.0,7.0,5,13.000000
0297fc1e,True,True,True,True,True,True,False,True,True,False,...,404.0,6.0,95.0,1.0,4.0,21.0,29.0,0.0,89,6.370787
2d681dd8,True,True,True,True,True,True,True,False,True,False,...,13.0,1.0,2.0,0.0,5.0,1.0,1.0,2.0,2,13.000000
cccea85e,True,True,True,True,True,True,True,False,True,False,...,739.0,1.0,22.0,0.0,7.0,20.0,15.0,26.0,17,49.176471


In [32]:
user_features.to_csv('data/features_basicas.csv', sep=',')